In [17]:
from scipy.optimize import minimize
import numpy as np
# from scipy.weave import inline
from random import gauss
import scipy as sp

In [18]:
def center_image_fiducials(dimensions,fiducials):

    ''' Centers image fiducial points using image dimensions  '''

    for i in range(np.shape(fiducials)[0]):
        fiducials[i,0] = fiducials[i,0] - (dimensions[0] / 2)
        fiducials[i,1] = (dimensions[1] / 2) - fiducials[i,1]

    return fiducials

In [19]:
def colin(params, xyz_a):

	# Unwrap params
	kappa, phi, omega, xs, ys, zs, f = params

	omega = float(omega)
	phi = float(phi) + 0.5*np.pi
	kappa = float(kappa)
	xs = float(xs)
	ys = float(ys)
	zs = float(zs)
	f = float(f)

	# -- utils
	co = np.cos(omega)
	so = np.sin(omega)
	cp = np.cos(phi)
	sp = np.sin(phi)
	ck = np.cos(kappa)
	sk = np.sin(kappa)

	a1 =  cp*ck+sp*so*sk
	b1 =  cp*sk+sp*so*ck
	c1 =  sp*co
	a2 = -co*sk
	b2 =  co*ck
	c2 =  so
	a3 =  sp*ck+cp*so*sk
	b3 =  sp*sk-cp*so*ck
	c3 =  cp*co

	ynum  = a1*(xyz_a[:,0]-xs)+b1*(xyz_a[:,1]-ys)+c1*(xyz_a[:,2]-zs)
	xnum  = a2*(xyz_a[:,0]-xs)+b2*(xyz_a[:,1]-ys)+c2*(xyz_a[:,2]-zs)
	denom = a3*(xyz_a[:,0]-xs)+b3*(xyz_a[:,1]-ys)+c3*(xyz_a[:,2]-zs)

	xx = -f*xnum/denom
	yy = f*ynum/denom

	return np.vstack([xx,yy]).T

        '''	
	# Get number of fiducial points
	N = int(xyz_a.shape[0])

	# Initialize the result vector
	colin_xy = np.zeros((N,2))

	# Define c code that will evaluate the functions
	code = """

		double sinp = sinf(phi);
		double cosp = cosf(phi);
		double sino = sinf(omega);
		double coso = cosf(omega);
		double sink = sinf(kappa);
		double cosk = cosf(kappa);

		double a1 = cosp*cosk;
		double b1 = coso*sink + sino*sinp*cosk;
		double c1 = sino*sink - coso*sinp*cosk;
		double a2 = -1 * cosp*sink;
		double b2 = coso*cosk - sino*sinp*sink;
		double c2 = sino*cosk + coso*sinp*sink;
		double a3 = sinp;
		double b3 = -1*sino*cosp;
		double c3 = cosp*coso;

		int j;
		int k;
		double denom;

		for(int i = 0; i < N; i++)
		{
			j = i*2;
			k = i*3;
			denom = (a3*(xyz_a[k]-xs) + b3*(xyz_a[(k+1)]-ys) + c3*(xyz_a[(k+2)]-zs));

			colin_xy[j] = -1.0*f*(a1*(xyz_a[k]-xs) + b1*(xyz_a[(k+1)]-ys) + c1*(xyz_a[(k+2)]-zs))/denom;
			colin_xy[(j+1)] = -1.0*f*(a2*(xyz_a[k]-xs) + b2*(xyz_a[(k+1)]-ys) + c2*(xyz_a[(k+2)]-zs))/denom;
		}
		return_val = 1;
	"""

	# Use scipy.weave.inline to run the c code
	res = inline(code, ['colin_xy', 'omega', 'phi', 'kappa', 'xs', \
		'ys', 'zs', 'f', 'xyz_a', 'N'], headers = ['<math.h>'], \
		compiler = 'gcc')
	
	# Return the pixel (x,y) positions
	return colin_xy
	'''


In [21]:
def fullfunc(params,xyz_s,xy_t):

    ''' Find the sum of squares difference '''
    omega, phi, kappa, xs, ys, zs, f = params


    if (omega<0.0) or (omega>=2.0*np.pi):
        return 1e9+omega**4
    elif (phi<-0.5*np.pi) or (phi>=0.5*np.pi):
        return 1e9+phi**4
    elif (kappa<0.0) or (kappa>=2.0*np.pi):
        return 1e9+kappa**4
    elif zs<0.0:
        return 1e9+zs**4
    elif f<0.0:
        return 1e9+f**4
#     elif (np.abs(params[3] - 977119)>1000) or \
#             (np.abs(params[4] - 210445)>1000):
#         return 1e9 + xs**2

    colin_xy = 1.0*colin(params,xyz_s.astype(float))
    diff = ((colin_xy - xy_t)**2).sum()

    return diff

In [22]:
def call(params,xyz_s,xy_t):

    ''' Guess parameters near start and brute-force minimize '''
    start = params

    res = minimize(fullfunc, start,args=(xyz_s,xy_t), \
        method = 'Nelder-Mead', \
        options={'maxfev': 10000, 'maxiter': 10000})

    return res

In [296]:
guess = np.array([4.48603184, -5.75616093e-02, 0.0115, 987818.3984021898, 214563.46676424053, 800, 3000])
# guess = np.array([2.75, -0.5, 0.01, 987818.3984021898, 214563.46676424053, 1100, 900])
# score, params = optimize.run("lwir", num_iter=100, params=guess)

In [297]:
# xyz_s = fiducials.lidar_fiducials('BoA')

In [298]:
lidar_fiducials = np.array([
                            [988224.09, 211951.573,1494.756662], #Empire state building
                            [980598.406, 199043.071,1750.127224], #WTC
                            [987656.616, 211766.233,493.89], # 1250 Broadway
                            [983564.98, 199358.775,591.406796], # Marshall courthouse
                            [987342.468, 212511.054,380.69], #  112 West 34th St
                            [988596.086, 211789.785,255.31], # 347 5th Ave
                            [988287.232, 213228.734,488.716947]]) # 66 W 38th St

In [299]:
xyz_s = lidar_fiducials

In [300]:
fiducials = np.array([
                        [621, 305],#Empire state building
                        [1683, 936],#WTC
                        [1185, 1400], # 1250 Broadway
                        [1217, 1143], # Marshall courthouse
                        [1860, 1637], #  112 West 34th St
                        [211, 1704], # 347 5th Ave
                        [814, 1811]])# 66 W 38th St

dimensions = np.array([1918, 2560])
fiducials = center_image_fiducials(dimensions,fiducials)

In [301]:
xy_t = fiducials

In [302]:
min_score = 100000000000000
num_iter = 100
params = guess

In [303]:
for i in range(0, num_iter):
    result = call(params,xyz_s,xy_t)
    print "params, score", result.x, result.fun
    if (result.fun < min_score):# and (result.x[3] < 980491):
        min_score = result.fun
        params = result.x

params, score [  4.48723386e+00  -5.75616171e-02   1.14226002e-02   9.88501080e+05
   2.14474001e+05   7.93662678e+02   2.84534724e+03] 1577.1864558
params, score [  4.48723386e+00  -5.75616144e-02   1.14226011e-02   9.88501080e+05
   2.14474001e+05   7.93662680e+02   2.84534724e+03] 1577.18645579
params, score [  4.48723386e+00  -5.75616144e-02   1.14226011e-02   9.88501080e+05
   2.14474001e+05   7.93662680e+02   2.84534724e+03] 1577.18645579
params, score [  4.48723386e+00  -5.75616144e-02   1.14226011e-02   9.88501080e+05
   2.14474001e+05   7.93662680e+02   2.84534724e+03] 1577.18645579
params, score [  4.48723386e+00  -5.75616144e-02   1.14226011e-02   9.88501080e+05
   2.14474001e+05   7.93662680e+02   2.84534724e+03] 1577.18645579
params, score [  4.48723386e+00  -5.75616144e-02   1.14226011e-02   9.88501080e+05
   2.14474001e+05   7.93662680e+02   2.84534724e+03] 1577.18645579
params, score [  4.48723386e+00  -5.75616144e-02   1.14226011e-02   9.88501080e+05
   2.14474001e+05 

In [304]:
print("score  = {0}\nparams = {1}".format(min_score,params))

score  = 1577.18645579
params = [  4.48723386e+00  -5.75616144e-02   1.14226011e-02   9.88501080e+05
   2.14474001e+05   7.93662680e+02   2.84534724e+03]
